In [43]:
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import operator
import math
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Steven\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Process data files

In [2]:
def eval_file(file):
    f = open(file, "r", encoding="utf-8")
    lst = []
    for line in f:
        obj = eval(line)
        lst.append(obj)
    json.dumps(lst)
    return lst

In [3]:
bundle_data = eval_file("../data/bundle_data.json")
users_items = eval_file("../data/australian_users_items.json")
steam_games = eval_file("../data/steam_games.json")

In [4]:
bundle_df = json_normalize(bundle_data, "items", ["bundle_final_price", "bundle_url", "bundle_price", "bundle_name", "bundle_id"])
steam_games_df = pd.DataFrame(data=steam_games)

C:\Users\Steven\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


# Data Preprocessing

In [78]:
def clean_bundle_data():    
    clean_bundle_df = bundle_df[bundle_df["genre"]!=""]
    return clean_bundle_df

In [79]:
def clean_games_data():
    steam_games_df.drop(labels=["app_name", "tags", "metascore"], axis=1, inplace=True)
    steam_games_df.dropna(subset=["title"], inplace=True)

In [80]:
def remove_idle_users():
    for i in users_items:
        if i["items_count"]==0:
            users_items.remove(i)

In [81]:
def filter_by_playtime(user_id, playtime):
    user_data = {}
    
    for user in users_items:
        if user["user_id"]==user_id:
            user_data = user
            break
    
    filtered_df = json_normalize(user_data, "items", ["user_id", "steam_id", "user_url"])
    filtered_df = filtered_df[filtered_df["playtime_forever"] > playtime]
    
    return filtered_df

In [82]:
def extract_genres_data_from_bundles(): 
    
    clean_bundle_df = clean_bundle_data()
    
    filter_genre_from_bundle = {}
    for id_ in clean_bundle_df["bundle_id"].unique():
        words = ""
        for genres in clean_bundle_df[clean_bundle_df["bundle_id"]==id_]["genre"]:
            for genre in genres.split(","):
                if genre not in words:
                    words = words + genre + " "
        filter_genre_from_bundle[id_] = words[:-1]
        
    return filter_genre_from_bundle

In [30]:
remove_idle_users()

In [26]:
clean_games_data()

In [29]:
steam_games_df.head()

,publisher,genres,title,url,release_date,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN
5,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,NaN,http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd,Mixed


In [83]:
all_bundles_genres_data = extract_genres_data_from_bundles()

In [32]:
filtered_playtime_df = filter_by_playtime('76561197970982479', 30)

C:\Users\Steven\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


In [33]:
filtered_playtime_df

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,steam_id,user_url
8,300,Day of Defeat: Source,4733,0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
9,240,Counter-Strike: Source,1853,0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
10,3830,Psychonauts,333,0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
11,2630,Call of Duty 2,75,0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
12,3900,Sid Meier's Civilization IV,338,0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...
269,368230,Kingdom: Classic,83,0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
272,289130,Endless Legend,593,0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
273,268500,XCOM 2,1495,0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
275,379720,DOOM,793,0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [34]:
# filtered_playtime_df = pd.DataFrame()
# prev = 0
# for i in range(100, len(users_items[:40000]), 100):
#     filtered_playtime_df = pd.concat([filtered_playtime_df, filter_by_playtime(users_items[prev:i], 120)])
#     prev = i+1

In [84]:
final_bundle_data = pd.DataFrame.from_dict(data=all_bundles_genres_data, orient="index", columns=["genres"])

In [85]:
final_bundle_data

,genres
450,Adventure Indie RPG Casual Action
1437,Casual Free to Play Indie Simulation Sport...
1478,Casual Simulation
1482,Casual Indie Adventure Casual Strategy Action
1400,Casual Indie
...,...
582,Adventure Indie
584,Adventure Indie
586,Adventure Casual Indie
588,Action Adventure Casual Indie


# Recommend bundle to user based on most played genres

In [37]:
def get_all_genres_by_user(user):
    genre_count = {}
    for index, row in user.iterrows():
        if steam_games_df[steam_games_df["id"]==row["item_id"]].index.any()==False:
            continue
        game_genres = steam_games_df[steam_games_df["id"]==row["item_id"]]["genres"].values[0][0]
        for genre in game_genres.split(","):
            if genre not in genre_count:
                genre_count[genre] = 1
            else:
                genre_count[genre] = genre_count[genre] + 1

    return sorted(genre_count.items(), key=operator.itemgetter(1), reverse=True)

In [38]:
def parse_genres(genres):
    g = ""
    for genre in genres[:math.ceil(len(genres)*.75)]:
        g = g + genre[0] + " "
    return g[:-1]

In [39]:
all_user_genres = get_all_genres_by_user(filtered_playtime_df)

In [42]:
top_rated_genres = parse_genres(all_user_genres)

In [46]:
def get_bundle_similarity_scores(user_genres, all_bundles_genres_data):
    sim = {}
    ug = word_tokenize(user_genres)
    
    for id_ in list(all_bundles_genres_data.keys()):
        bundle_genres = word_tokenize(all_bundles_genres_data[id_])

        lst1 = []
        lst2 = []
        cos = 0
        vector = list(set().union(ug, bundle_genres))
        for i in vector:
            if i in ug:
                lst1.append(1)
            else:
                lst1.append(0)
                
            if i in bundle_genres:
                lst2.append(1)
            else:
                lst2.append(0)
            cos += lst1[-1]*lst2[-1]

        sim[id_] = cos / float((sum(lst1)*sum(lst2))**0.5)
    
    return sorted(sim.items(), key=operator.itemgetter(1), reverse=True)

In [47]:
sim = get_bundle_similarity_scores(top_rated_genres, all_bundles_genres_data)

In [48]:
sim

[('1206', 1.0),
 ('398', 1.0),
 ('396', 0.9258200997725514),
 ('505', 0.9258200997725514),
 ('364', 0.9258200997725514),
 ('399', 0.9258200997725514),
 ('482', 0.9258200997725514),
 ('957', 0.9258200997725514),
 ('804', 0.9258200997725514),
 ('403', 0.9258200997725514),
 ('542', 0.9258200997725514),
 ('356', 0.9258200997725514),
 ('450', 0.9128709291752769),
 ('1482', 0.9128709291752769),
 ('279', 0.9128709291752769),
 ('347', 0.9128709291752769),
 ('977', 0.9128709291752769),
 ('611', 0.9128709291752769),
 ('1250', 0.9128709291752769),
 ('434', 0.9128709291752769),
 ('990', 0.9128709291752769),
 ('1056', 0.9128709291752769),
 ('1054', 0.9128709291752769),
 ('1055', 0.9128709291752769),
 ('1040', 0.9128709291752769),
 ('949', 0.9128709291752769),
 ('920', 0.9128709291752769),
 ('854', 0.9128709291752769),
 ('728', 0.9128709291752769),
 ('665', 0.9128709291752769),
 ('153', 0.9128709291752769),
 ('260', 0.9128709291752769),
 ('350', 0.9128709291752769),
 ('433', 0.9128709291752769),
 ('

In [94]:
def get_bundle_recommendations(user_id, num_recommendations, sim, bundle_data=bundle_data):
    
    recommendations = []
    for id_ in sim[:num_recommendations]:
        for bundle in bundle_data:
            if bundle["bundle_id"]==id_[0]:
                recommendations.append(bundle)
                break
    
    return recommendations

In [95]:
recommendations = get_bundle_recommendations('76561197970982479', 3, sim)

In [98]:
recommendations

[{'bundle_final_price': '$55.40',
  'bundle_url': 'http://store.steampowered.com/bundle/1206/?utm_source=SteamDB&utm_medium=SteamDB&utm_campaign=SteamDB%20Bundles%20Page',
  'bundle_price': '$110.90',
  'bundle_name': 'Fatshark Bundle',
  'bundle_id': '1206',
  'items': [{'genre': 'Adventure, Casual, Indie',
    'item_id': '42140',
    'discounted_price': '$9.99',
    'item_url': 'http://store.steampowered.com/app/42140',
    'item_name': "Hamilton's Great Adventure"},
   {'genre': 'Adventure, Indie, Casual',
    'item_id': '42180',
    'discounted_price': '$0.99',
    'item_url': 'http://store.steampowered.com/app/42180',
    'item_name': "Hamilton's Great Adventure: Retro Fever"},
   {'genre': 'Action, Adventure, Indie, RPG, Strategy',
    'item_id': '42170,204731,204732',
    'discounted_price': '$19.99',
    'item_url': 'http://store.steampowered.com/app/42170,204731,204732',
    'item_name': "Krater - Collector's Edition"},
   {'genre': 'Action, Adventure, Strategy, RPG, Indie',
 